In [2]:
import torch
import os
import torchaudio
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

In [3]:
%%time
path_to_clips = "dataset/multilingual_spoken_words/mswc_microset_wav/pl/clips"
path_to_output = "dataset/multilingual_spoken_words/mswc_microset_wav/pl/clips_tensors"

transform = torchaudio.transforms.MelSpectrogram(n_mels=49, hop_length=401, power=0.75)

pbar = tqdm(os.listdir(path_to_clips))
for label in pbar:
    pbar.set_description(label)
    os.makedirs(f'{path_to_output}/{label}', exist_ok=True)
    for audio in os.listdir(path_to_clips + "/" + label):
        src = f'{path_to_clips}/{label}/{audio}'
        dest = f'{path_to_output}/{label}/{audio}'.replace(".wav", ".pt")
        waveform, _ = torchaudio.load(src)
        target = torch.zeros(49, 40)
        transformed = transform(waveform)[0]
        target[:, :min(transformed.shape[1], 40)] = transformed[:, :min(transformed.shape[1], 40)]
#         if spectrogram.shape != (49, 40):
#             display("atata " + f'{src} -> {spectrogram.shape}')
#         print(spectrogram)
#         break
        torch.save(target.cuda(), dest)

ubrana: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14525/14525 [10:54<00:00, 22.20it/s]

CPU times: user 1h 53s, sys: 1min 52s, total: 1h 2min 45s
Wall time: 10min 54s


In [10]:
path_to_wav = path_to_clips
path_to_tensors = path_to_output

def test_loading(func, is_tensors):
    path = path_to_tensors if is_tensors else path_to_wav
    for label in tqdm(os.listdir(path)):
        for audio in os.listdir(f'{path}/{label}'):
            filepath = f'{path}/{label}/{audio}'
            func(filepath)
            
def load_tensor(filepath):
    spectrogram = torch.load(filepath)
    return

def load_wav(filepath):
    waveform, _ = torchaudio.load(filepath)
    spectrogram = transform(waveform)
    return

In [12]:
%%time
test_loading(load_wav, is_tensors=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15900/15900 [08:08<00:00, 32.57it/s]

CPU times: user 44min 43s, sys: 45.1 s, total: 45min 28s
Wall time: 8min 8s


In [14]:
%%time
test_loading(load_tensor, is_tensors=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15900/15900 [05:21<00:00, 49.48it/s]

CPU times: user 2min 56s, sys: 1min 22s, total: 4min 18s
Wall time: 5min 21s
